Импортируем необходимые библиотеки:

In [2]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine
import plotly.express as px

Создадим строку подключения к БД и призведем само подключение к этой БД:

In [ ]:
db_config = {'user': 'praktikum_student', # имя пользователя
            'pwd': 'Sdf4$2;d-d30pp', # пароль
            'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
            'port': 6432, # порт подключения
            'db': 'data-analyst-zen-project-db'} # название базы данных

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                db_config['pwd'],
                                                db_config['host'],
                                                db_config['port'],
                                                db_config['db'])

engine = create_engine(connection_string) 

Выгрузим всю информацию из таблицы, в которой находятся данные необходимые для анализа:

In [3]:
query = '''SELECT * FROM dash_visits'''


dash_df = pd.io.sql.read_sql(query, con = engine) 

Проверим выгрузку:

In [4]:
dash_df.head()

,record_id,item_topic,source_topic,age_segment,dt,visits
0,1040597,Деньги,Авто,18-25,2019-09-24 18:32:00,3
1,1040598,Деньги,Авто,18-25,2019-09-24 18:35:00,1
2,1040599,Деньги,Авто,18-25,2019-09-24 18:54:00,4
3,1040600,Деньги,Авто,18-25,2019-09-24 18:55:00,17
4,1040601,Деньги,Авто,18-25,2019-09-24 18:56:00,27


In [5]:
dash_df.shape

(30745, 6)

Перезапишем полученные данные в другой текстовый файл, из которого будем брать данные для графиков дашборда:

In [6]:
dash_df.to_csv('dash_visits.csv', index=False)

Ссылка на дашборд на Tableau Public:<br>
https://public.tableau.com/views/_16716483209820/sheet4?:language=en-US&:display_count=n&:origin=viz_share_link

  ---------- Вспомогательные ячейки -----------------

Получим топ5 тем карточек:

In [19]:
pd.pivot_table(dash_df, index=['item_topic'], values='record_id',aggfunc='count').reset_index().rename(columns={'record_id':'qty'}).sort_values(by='qty', ascending=False).head()

,item_topic,qty
13,Отношения,1536
6,Интересные факты,1535
11,Наука,1505
14,Подборки,1456
15,Полезные советы,1424


Получим топ5 тем источников:

In [20]:
pd.pivot_table(dash_df, index=['source_topic'], values='record_id',aggfunc='count').reset_index().rename(columns={'record_id':'qty'}).sort_values(by='qty', ascending=False).head()

,source_topic,qty
20,Семейные отношения,1822
17,Россия,1687
5,Знаменитости,1650
12,Полезные советы,1578
15,Путешествия,1563


Получим топ5 возрастных категорий:

In [21]:
pd.pivot_table(dash_df, index=['age_segment'], values='record_id',aggfunc='count').reset_index().rename(columns={'record_id':'qty'}).sort_values(by='qty', ascending=False).head()

,age_segment,qty
0,18-25,7056
1,26-30,5875
2,31-35,5552
3,36-40,5105
4,41-45,3903


Вузуализируем колебания тем карточек в виде boxplot:

In [26]:


fig = px.box(dash_df, x='item_topic', y='visits')
fig.update_layout(yaxis_range=[0,30])
fig.update_layout(xaxis={'categoryorder':'total descending'})
fig.show()

Получим топ5 пар тема карточки - тема источника:

In [31]:
dash_df.groupby(['source_topic', 'item_topic'])['visits'].agg('sum').reset_index().sort_values(by='visits', ascending=False).head()

,source_topic,item_topic,visits
392,Путешествия,Рассказы,4587
436,Россия,Общество,3471
235,Кино,Наука,3279
443,Россия,Россия,2847
313,Полезные советы,Подборки,2795
